# Structure from Motion (SfM)
## A Complete Implementation Guide for Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

**Structure from Motion** recovers the 3D structure of a scene and the camera poses that observed it, using only a collection of 2D images. It is the backbone of systems like Google Street View, Photogrammetry software, and autonomous vehicle mapping.

---

## The SfM Pipeline

```
  Images
    │
    ▼
  Feature Detection & Description   (SIFT keypoints + descriptors)
    │
    ▼
  Feature Matching                  (cross-image correspondences)
    │
    ▼
  Geometric Verification            (RANSAC + Fundamental/Essential Matrix)
    │
    ▼
  Camera Pose Recovery              (decompose Essential Matrix → R, t)
    │
    ▼
  Triangulation                     (3D point cloud from 2D correspondences)
    │
    ▼
  Bundle Adjustment                 (jointly refine cameras + 3D points)
    │
    ▼
  3D Reconstruction                 (sparse point cloud + camera poses)
```

---

## Notebook Contents
| # | Topic |
|---|---|
| 1 | Setup & GPU check |
| 2 | The pinhole camera model |
| 3 | Feature detection with SIFT |
| 4 | Feature matching & ratio test |
| 5 | The Fundamental Matrix |
| 6 | The Essential Matrix |
| 7 | RANSAC for robust estimation |
| 8 | Camera pose recovery (R, t) |
| 9 | Triangulation |
| 10 | Incremental SfM on real images |
| 11 | Bundle Adjustment |
| 12 | 3D Point Cloud Visualization |

## 1. Setup & GPU Check

In [ ]:
%%capture
!pip install opencv-contrib-python-headless --quiet   # SIFT + extra matchers
!pip install scipy matplotlib numpy tqdm --quiet
!pip install open3d --quiet                           # 3D point cloud viewer
!pip install plotly --quiet                           # interactive 3D plots
print('Done')

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix
from itertools import combinations
import os, warnings, time
from pathlib import Path
from tqdm.notebook import tqdm
from PIL import Image
warnings.filterwarnings('ignore')

# Reproducibility
np.random.seed(42)

# Check cv2 build (must have SIFT)
print(f'OpenCV version : {cv2.__version__}')
try:
    sift_test = cv2.SIFT_create()
    print('SIFT           : ✅ available')
except:
    print('SIFT           : ❌ not available — reinstall opencv-contrib-python-headless')

import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device         : {DEVICE}')
if DEVICE == 'cuda':
    print(f'GPU            : {torch.cuda.get_device_name(0)}')

## 2. The Pinhole Camera Model

Every SfM system starts with a mathematical model of how a 3D point projects onto a 2D image plane.

### 2.1 Projection Formula

A 3D world point $\mathbf{X} = [X, Y, Z]^T$ projects to a 2D image pixel $\mathbf{x} = [u, v]^T$ via:

$$\lambda \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = \mathbf{K} \begin{bmatrix} \mathbf{R} & \mathbf{t} \end{bmatrix} \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}$$

where $\lambda$ is the depth, $\mathbf{K}$ is the **intrinsic matrix**, and $[\mathbf{R} | \mathbf{t}]$ is the **extrinsic matrix**.

### 2.2 The Intrinsic Matrix K

$$\mathbf{K} = \begin{bmatrix} f_x & s & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}$$

- $f_x, f_y$ — focal lengths in pixels (typically equal for square pixels)
- $c_x, c_y$ — principal point (usually image center)
- $s$ — skew (almost always 0)

### 2.3 The Extrinsic Matrix [R | t]

Transforms points from **world coordinates** into **camera coordinates**:

$$\mathbf{X}_{cam} = \mathbf{R} \mathbf{X}_{world} + \mathbf{t}$$

- $\mathbf{R}$ — 3×3 rotation matrix (camera orientation)
- $\mathbf{t}$ — 3×1 translation vector (camera position in world, negated)

**Goal of SfM:** Recover $\mathbf{K}$, $\mathbf{R}_i$, $\mathbf{t}_i$ for each camera $i$, and the 3D positions $\mathbf{X}_j$ of all scene points $j$.

In [ ]:
# ── Camera model implementation ───────────────────────────────────

def build_K(fx, fy, cx, cy, skew=0.0):
    """Build the 3×3 intrinsic matrix K."""
    return np.array([[fx, skew, cx],
                     [ 0,   fy, cy],
                     [ 0,    0,  1]], dtype=np.float64)


def project_points(X_world, K, R, t):
    """
    Project 3D world points into 2D image pixels.

    Args:
        X_world : (N, 3) world coordinates
        K       : (3, 3) intrinsic matrix
        R       : (3, 3) rotation matrix
        t       : (3,)   translation vector
    Returns:
        pts2d   : (N, 2) pixel coordinates [u, v]
        depths  : (N,)   depth values (Z in camera frame)
    """
    # Transform to camera coordinates
    X_cam = (R @ X_world.T + t.reshape(3,1)).T   # (N, 3)
    depths = X_cam[:, 2]

    # Perspective divide → normalised image coords
    x_norm = X_cam[:, 0] / (X_cam[:, 2] + 1e-10)
    y_norm = X_cam[:, 1] / (X_cam[:, 2] + 1e-10)

    # Apply K
    u = K[0,0] * x_norm + K[0,2]
    v = K[1,1] * y_norm + K[1,2]

    return np.stack([u, v], axis=-1), depths


def unproject_pixel(px, K):
    """
    Convert pixel (u,v) to normalised camera ray (x_n, y_n, 1).
    This is the inverse of K: x_norm = K^{-1} * [u, v, 1]^T
    """
    K_inv = np.linalg.inv(K)
    px_h  = np.array([px[0], px[1], 1.0])
    return K_inv @ px_h


# ── Visualise projection ─────────────────────────────────────────

# Example: synthetic 3D scene
np.random.seed(0)
N_pts   = 50
X_world = np.random.randn(N_pts, 3) * 2.0
X_world[:, 2] += 8.0         # place scene ~8 m in front of camera

# Camera looking straight down -Z (identity rotation, at origin)
K_demo = build_K(fx=800, fy=800, cx=320, cy=240)
R_demo = np.eye(3)
t_demo = np.zeros(3)

pts2d, depths = project_points(X_world, K_demo, R_demo, t_demo)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

ax = axes[0]
ax.scatter(X_world[:,0], X_world[:,2], c=depths, cmap='plasma', s=40, zorder=3)
ax.set_xlabel('X (m)'); ax.set_ylabel('Z depth (m)')
ax.set_title('3D Scene (side view)', fontweight='bold')
ax.axvline(0, color='k', lw=1, ls='--'); ax.axhline(0, color='k', lw=1, ls='--')
ax.grid(alpha=0.3)

ax = axes[1]
valid = (depths > 0) & (pts2d[:,0] >= 0) & (pts2d[:,0] < 640) & \
        (pts2d[:,1] >= 0) & (pts2d[:,1] < 480)
sc = ax.scatter(pts2d[valid,0], pts2d[valid,1], c=depths[valid], cmap='plasma', s=40)
ax.set_xlim(0, 640); ax.set_ylim(480, 0)
ax.set_xlabel('u (pixels)'); ax.set_ylabel('v (pixels)')
ax.set_title('Projected Image (640×480)', fontweight='bold')
ax.add_patch(plt.Rectangle((0,0), 640, 480, fill=False, ec='red', lw=2))
plt.colorbar(sc, ax=ax, label='Depth (m)')
ax.grid(alpha=0.3)

plt.suptitle('Pinhole Camera Projection', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print(f'K =\n{K_demo}')
print(f'\nProjected {valid.sum()}/{N_pts} points into the image frame')
print(f'Depth range: [{depths.min():.2f}, {depths.max():.2f}] m')

## 3. Feature Detection with SIFT

SfM requires **repeatable, distinctive keypoints** that can be matched across different viewpoints and lighting conditions.

**SIFT (Scale-Invariant Feature Transform)** is the gold standard:
- Finds keypoints at multiple scales using a **Difference of Gaussians (DoG)** pyramid
- Each keypoint gets a **128-dimensional descriptor** encoding local gradient orientations
- Invariant to: scale, rotation, illumination, and moderate viewpoint changes

### DoG Scale-Space

$$D(x, y, \sigma) = L(x, y, k\sigma) - L(x, y, \sigma)$$

where $L(x, y, \sigma) = G(x, y, \sigma) * I(x, y)$ is the image convolved with a Gaussian at scale $\sigma$. Keypoints are local extrema (maxima/minima) in this 3D scale-space.

In [ ]:
# ── Generate synthetic test scene ────────────────────────────────
# We create a textured planar scene viewed from two angles.
# Using synthetic data means we have perfect ground truth for validation.

def make_textured_image(H=480, W=640, seed=0):
    """Synthetic scene: checkerboard + random circles + gradient."""
    rng = np.random.default_rng(seed)
    img = np.zeros((H, W, 3), dtype=np.uint8)

    # Checkerboard background
    sq = 40
    for i in range(0, H, sq):
        for j in range(0, W, sq):
            if (i//sq + j//sq) % 2 == 0:
                img[i:i+sq, j:j+sq] = [200, 200, 200]
            else:
                img[i:i+sq, j:j+sq] = [60, 60, 60]

    # Coloured circles
    for _ in range(30):
        x, y = rng.integers(50, W-50), rng.integers(50, H-50)
        r    = rng.integers(15, 50)
        c    = rng.integers(80, 255, 3)
        cv2.circle(img, (x, y), r, c.tolist(), -1)

    # Text features for distinctiveness
    for i, ch in enumerate('SfM DEMO'):
        cv2.putText(img, ch, (60 + i*65, 60), cv2.FONT_HERSHEY_SIMPLEX,
                    1.8, (255,255,0), 3)

    return img


def make_view_pair(img, angle_deg=15.0, scale=1.0, tx=40, ty=20):
    """
    Simulate a second camera view via a homography (planar scene assumption).
    Returns (img1, img2, H_gt) where H_gt is the ground-truth homography.
    """
    H, W = img.shape[:2]
    cx, cy = W/2, H/2

    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)

    # Rotation + scale + translation homography
    H_gt = np.array([
        [scale*cos_a, -scale*sin_a, tx + cx*(1 - scale*cos_a) + cy*scale*sin_a],
        [scale*sin_a,  scale*cos_a, ty + cy*(1 - scale*cos_a) - cx*scale*sin_a],
        [0,            0,           1]
    ], dtype=np.float64)

    img2 = cv2.warpPerspective(img, H_gt, (W, H),
                                flags=cv2.INTER_LINEAR,
                                borderMode=cv2.BORDER_CONSTANT,
                                borderValue=(128,128,128))
    return img, img2, H_gt


# Create images
base_img = make_textured_image(seed=5)
img1, img2, H_true = make_view_pair(base_img, angle_deg=12, scale=0.92, tx=50, ty=30)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].imshow(img1); axes[0].set_title('Image 1 — Reference view', fontweight='bold'); axes[0].axis('off')
axes[1].imshow(img2); axes[1].set_title('Image 2 — Rotated / translated view', fontweight='bold'); axes[1].axis('off')
plt.suptitle('Synthetic Image Pair', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ── SIFT Feature Detection ────────────────────────────────────────

def detect_features(img, n_features=2000):
    """
    Detect SIFT keypoints and compute descriptors.

    Returns:
        kps   : list of cv2.KeyPoint
        descs : (N, 128) float32 descriptor array
        pts   : (N, 2) float32 pixel coordinates
    """
    sift  = cv2.SIFT_create(nfeatures=n_features)
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim == 3 else img
    kps, descs = sift.detectAndCompute(gray, None)

    if descs is None:
        return [], np.empty((0,128), np.float32), np.empty((0,2))

    pts = np.array([kp.pt for kp in kps], dtype=np.float32)
    return kps, descs, pts


def visualise_keypoints(img, kps, title='Keypoints', max_show=500):
    """Draw keypoints with scale and orientation circles."""
    vis = cv2.drawKeypoints(img, kps[:max_show], None,
                             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return cv2.cvtColor(vis, cv2.COLOR_BGR2RGB) if vis.ndim == 3 else vis


kps1, descs1, pts1 = detect_features(img1)
kps2, descs2, pts2 = detect_features(img2)

print(f'Image 1: {len(kps1)} keypoints detected')
print(f'Image 2: {len(kps2)} keypoints detected')
print(f'Descriptor shape: {descs1.shape}  (N × 128)')

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(visualise_keypoints(img1, kps1))
axes[0].set_title(f'Image 1 — {len(kps1)} SIFT keypoints\n(circles = scale, line = orientation)', fontweight='bold')
axes[0].axis('off')
axes[1].imshow(visualise_keypoints(img2, kps2))
axes[1].set_title(f'Image 2 — {len(kps2)} SIFT keypoints', fontweight='bold')
axes[1].axis('off')
plt.suptitle('SIFT Feature Detection', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

# Distribution of keypoint scales
scales1 = [kp.size for kp in kps1]
fig, ax = plt.subplots(figsize=(8, 3))
ax.hist(scales1, bins=40, color='steelblue', edgecolor='white', lw=0.5)
ax.set_xlabel('Keypoint scale (pixels)')
ax.set_ylabel('Count')
ax.set_title('SIFT Scale Distribution — keypoints detected at multiple scales', fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Feature Matching & Lowe's Ratio Test

To match keypoints across images, we find the **nearest neighbour** in descriptor space using Euclidean distance. However, naive nearest-neighbour matching produces many false matches.

### Lowe's Ratio Test

For each descriptor in image 1, find the two nearest descriptors in image 2. Accept the match only if:

$$\frac{d_1}{d_2} < \tau \quad (\text{typically } \tau = 0.75)$$

where $d_1$ is the distance to the best match and $d_2$ is the distance to the second-best match.

**Intuition:** If the best match is significantly closer than the second-best, it is likely a true match. If both distances are similar, the descriptor is ambiguous and the match is unreliable.

In [ ]:
def match_features(descs1, descs2, ratio_thresh=0.75):
    """
    Match SIFT descriptors using FLANN + Lowe's ratio test.

    Returns:
        good_matches : list of cv2.DMatch (filtered)
        all_matches  : list of cv2.DMatch (before ratio test)
    """
    # FLANN (Fast Library for Approximate Nearest Neighbors)
    # Much faster than brute-force for large descriptor sets
    FLANN_INDEX_KDTREE = 1
    index_params  = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # knnMatch: return the 2 nearest neighbours for ratio test
    matches_knn = flann.knnMatch(descs1, descs2, k=2)

    good_matches = []
    for m, n in matches_knn:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)

    return good_matches, matches_knn


def draw_matches_vis(img1, kps1, img2, kps2, matches, max_show=80, title=''):
    """Draw match lines between two images."""
    vis = cv2.drawMatches(img1, kps1, img2, kps2,
                           matches[:max_show], None,
                           flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
                           matchColor=(0, 255, 0),
                           singlePointColor=(200, 200, 200))
    return cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)


good_matches, all_matches = match_features(descs1, descs2, ratio_thresh=0.75)

print(f'Total matches (before ratio test): {len(all_matches)}')
print(f'Good matches  (after  ratio test): {len(good_matches)}')
print(f'Rejection rate: {100*(1 - len(good_matches)/len(all_matches)):.1f}%')

# Visualise
fig, axes = plt.subplots(1, 1, figsize=(16, 6))
axes.imshow(draw_matches_vis(img1, kps1, img2, kps2, good_matches))
axes.set_title(f'SIFT Matches after Lowe\'s Ratio Test — {len(good_matches)} matches shown',
               fontweight='bold')
axes.axis('off')
plt.tight_layout()
plt.show()

# Show ratio test effect
ratios = [m[0].distance / m[1].distance for m in all_matches if len(m) == 2]
fig, ax = plt.subplots(figsize=(8, 3.5))
ax.hist(ratios, bins=50, color='steelblue', edgecolor='white', lw=0.5)
ax.axvline(0.75, color='red', lw=2, ls='--', label='Ratio threshold = 0.75')
ax.fill_betweenx([0, ax.get_ylim()[1] if ax.get_ylim()[1] > 0 else 200],
                  0, 0.75, alpha=0.15, color='green', label='Accepted')
ax.set_xlabel('Distance ratio d₁/d₂')
ax.set_ylabel('Count')
ax.set_title("Lowe's Ratio Test Distribution", fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 5. The Fundamental Matrix

Even after the ratio test, matches still contain outliers. The **Fundamental Matrix** $\mathbf{F}$ encodes the **epipolar constraint** — a geometric consistency check between any two views.

### The Epipolar Constraint

For any true correspondence $(\mathbf{x}, \mathbf{x}')$ between image 1 and image 2:

$$\mathbf{x}'^T \mathbf{F} \mathbf{x} = 0$$

This means that for a point $\mathbf{x}$ in image 1, its match in image 2 must lie on the **epipolar line** $\mathbf{l}' = \mathbf{F}\mathbf{x}$.

### Geometry: Epipoles and Epipolar Lines

- **Epipole $\mathbf{e}$**: The projection of camera 2's centre into image 1 (and vice versa)
- **Epipolar plane**: The plane through the two camera centres and a 3D point
- **Epipolar line**: The intersection of the epipolar plane with each image plane

All epipolar lines in image 1 pass through the epipole $\mathbf{e}_1$; all lines in image 2 pass through $\mathbf{e}_2$.

### Properties of F
- 3×3 matrix, rank 2 (det(**F**) = 0)
- 7 degrees of freedom (9 elements, scale-invariant, rank-2 constraint)
- Can be estimated from **7 or 8 point correspondences**

In [ ]:
def extract_match_points(kps1, kps2, matches):
    """Extract matched pixel coordinates from keypoints and match list."""
    pts1 = np.float32([kps1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kps2[m.trainIdx].pt for m in matches])
    return pts1, pts2


def epipolar_distance(F, pts1, pts2):
    """
    Compute symmetric epipolar distance for each correspondence.
    A perfect match has distance = 0.

    Symmetric distance = d(x', Fx)^2 + d(x, F^T x')^2
    where d(p, l) is the point-to-line distance.
    """
    pts1_h = np.column_stack([pts1, np.ones(len(pts1))])
    pts2_h = np.column_stack([pts2, np.ones(len(pts2))])

    # Epipolar lines in image 2: l2 = F @ x1
    l2 = (F @ pts1_h.T).T          # (N, 3)
    # Epipolar lines in image 1: l1 = F^T @ x2
    l1 = (F.T @ pts2_h.T).T        # (N, 3)

    # Point-to-line distances
    def pt_line_dist(pts_h, lines):
        num = np.abs(np.sum(pts_h * lines, axis=1))
        den = np.sqrt(lines[:,0]**2 + lines[:,1]**2 + 1e-10)
        return num / den

    d1 = pt_line_dist(pts1_h, l1)
    d2 = pt_line_dist(pts2_h, l2)
    return d1 + d2


# Extract matched points
match_pts1, match_pts2 = extract_match_points(kps1, kps2, good_matches)

# Estimate F with RANSAC (automatically discards outliers)
F, inlier_mask = cv2.findFundamentalMat(
    match_pts1, match_pts2,
    method=cv2.FM_RANSAC,
    ransacReprojThreshold=3.0,   # pixels
    confidence=0.999
)

inliers = inlier_mask.ravel().astype(bool)
pts1_in = match_pts1[inliers]
pts2_in = match_pts2[inliers]

print(f'Fundamental Matrix F:')
print(np.round(F, 6))
print(f'\nRank of F: {np.linalg.matrix_rank(F)}  (should be 2)')
print(f'det(F)   : {np.linalg.det(F):.2e}  (should be ~0)')
print(f'\nInliers : {inliers.sum()} / {len(good_matches)}')

epi_dists = epipolar_distance(F, pts1_in, pts2_in)
print(f'Mean symmetric epipolar distance (inliers): {epi_dists.mean():.3f} px')

# ── Draw epipolar lines ──────────────────────────────────────────
def draw_epipolar_lines(img1, img2, F, pts1, pts2, n_lines=12):
    """Draw corresponding epipolar lines on both images."""
    H, W = img1.shape[:2]
    vis1 = img1.copy(); vis2 = img2.copy()

    idx    = np.random.choice(len(pts1), min(n_lines, len(pts1)), replace=False)
    colors = plt.cm.hsv(np.linspace(0, 1, len(idx)))[:, :3]
    colors = (colors * 255).astype(int)

    for i, (ii, col) in enumerate(zip(idx, colors)):
        c = tuple(col.tolist())
        p1, p2 = tuple(pts1[ii].astype(int)), tuple(pts2[ii].astype(int))

        # Epipolar line in image 2
        l2 = F @ np.array([pts1[ii,0], pts1[ii,1], 1.0])
        x0, y0 = 0, int(-l2[2] / (l2[1] + 1e-10))
        x1, y1 = W, int(-(l2[2] + l2[0]*W) / (l2[1] + 1e-10))
        cv2.line(vis2, (x0,y0), (x1,y1), c, 1)
        cv2.circle(vis2, p2, 6, c, -1)

        # Epipolar line in image 1
        l1 = F.T @ np.array([pts2[ii,0], pts2[ii,1], 1.0])
        x0, y0 = 0, int(-l1[2] / (l1[1] + 1e-10))
        x1, y1 = W, int(-(l1[2] + l1[0]*W) / (l1[1] + 1e-10))
        cv2.line(vis1, (x0,y0), (x1,y1), c, 1)
        cv2.circle(vis1, p1, 6, c, -1)

    return cv2.cvtColor(vis1, cv2.COLOR_BGR2RGB), cv2.cvtColor(vis2, cv2.COLOR_BGR2RGB)


epi1, epi2 = draw_epipolar_lines(img1, img2, F, pts1_in, pts2_in, n_lines=15)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
axes[0].imshow(epi1); axes[0].set_title('Epipolar Lines — Image 1\n(each colour = one correspondence)', fontweight='bold'); axes[0].axis('off')
axes[1].imshow(epi2); axes[1].set_title('Epipolar Lines — Image 2\n(point must lie on coloured line)', fontweight='bold'); axes[1].axis('off')
plt.suptitle('Epipolar Geometry from Fundamental Matrix', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. The Essential Matrix

The **Essential Matrix** $\mathbf{E}$ is the calibrated version of $\mathbf{F}$. While $\mathbf{F}$ works in pixel coordinates, $\mathbf{E}$ works in **normalised camera coordinates** (after removing the effect of $\mathbf{K}$):

$$\mathbf{E} = \mathbf{K}'^T \mathbf{F} \mathbf{K}$$

The constraint becomes:

$$\hat{\mathbf{x}}'^T \mathbf{E} \hat{\mathbf{x}} = 0$$

where $\hat{\mathbf{x}} = \mathbf{K}^{-1}\mathbf{x}$ are normalised coordinates.

### Why E is more useful than F

$\mathbf{E}$ encodes **metric information** — it directly gives us the rotation $\mathbf{R}$ and (unit) translation $\mathbf{t}$ between cameras via SVD:

$$\mathbf{E} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T \quad\text{where}\quad \mathbf{\Sigma} = \text{diag}(\sigma, \sigma, 0)$$

Properties:
- Two singular values are equal, one is zero
- 5 degrees of freedom (vs 7 for F)
- Can be estimated from just **5 point correspondences**

In [ ]:
# ── Camera intrinsics ────────────────────────────────────────────
# For our synthetic images (640×480)
H_img, W_img = img1.shape[:2]
focal = max(H_img, W_img) * 1.2   # reasonable estimate: ~768 px
K = build_K(fx=focal, fy=focal, cx=W_img/2, cy=H_img/2)

print(f'Camera Intrinsics K:')
print(K)

# ── Compute Essential Matrix from F and K ─────────────────────────
E_from_F = K.T @ F @ K

# Or estimate E directly (5-point algorithm via RANSAC)
E_direct, e_mask = cv2.findEssentialMat(
    pts1_in, pts2_in, K,
    method=cv2.RANSAC,
    prob=0.999,
    threshold=1.0
)

# ── Validate E using SVD ─────────────────────────────────────────
def check_essential(E):
    """Verify E properties: rank 2, two equal singular values."""
    U, sv, Vt = np.linalg.svd(E)
    sv_norm   = sv / sv[0]  # normalise
    print(f'  Singular values (normalised): {sv_norm.round(4)}')
    print(f'  Ratio σ1/σ2: {sv[0]/sv[1]:.4f}  (ideal = 1.000)')
    print(f'  σ3:          {sv[2]:.6f}  (ideal = 0.000)')
    return U, sv, Vt

print('\nEssential Matrix (from F):')
U1, sv1, Vt1 = check_essential(E_from_F)

print('\nEssential Matrix (direct 5-point):')
U2, sv2, Vt2 = check_essential(E_direct)

# Use the directly estimated E going forward
E = E_direct
e_inliers = e_mask.ravel().astype(bool) if e_mask is not None else np.ones(len(pts1_in), bool)
pts1_e = pts1_in[e_inliers]
pts2_e = pts2_in[e_inliers]
print(f'\nEssential matrix inliers: {e_inliers.sum()} / {len(pts1_in)}')

## 7. RANSAC — Robust Estimation

Both `findFundamentalMat` and `findEssentialMat` internally use **RANSAC (Random Sample Consensus)** to robustly estimate the matrices in the presence of outlier matches.

### RANSAC Algorithm

```
for i = 1 to N_iterations:
    1. Sample the minimum number of points (e.g. 8 for F, 5 for E)
    2. Fit the model (F or E) to this minimal sample
    3. Count inliers: points where error < threshold
    4. If inlier count > best so far, save this model

Refit the model using all inliers of the best hypothesis
```

### Number of Iterations Required

To guarantee finding a good model with probability $p$ when the outlier ratio is $\epsilon$ and sample size is $s$:

$$N = \frac{\log(1 - p)}{\log(1 - (1 - \epsilon)^s)}$$

In [ ]:
def ransac_iterations(outlier_ratio, sample_size, confidence=0.999):
    """Compute required RANSAC iterations."""
    p_all_inliers = (1 - outlier_ratio) ** sample_size
    if p_all_inliers >= 1.0:
        return 1
    return int(np.ceil(np.log(1 - confidence) / np.log(1 - p_all_inliers + 1e-10)))


# ── Visualise RANSAC iteration requirements ───────────────────────
outlier_ratios  = np.linspace(0.01, 0.90, 100)
sample_sizes    = [4, 5, 7, 8]
labels          = ['Homography (4pt)', 'Essential E (5pt)', 'F 7-pt', 'F 8-pt']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for s, label in zip(sample_sizes, labels):
    iters = [ransac_iterations(e, s) for e in outlier_ratios]
    axes[0].semilogy(outlier_ratios * 100, iters, lw=2, label=label)

axes[0].set_xlabel('Outlier ratio (%)')
axes[0].set_ylabel('Required iterations (log scale)')
axes[0].set_title('RANSAC Iterations vs Outlier Ratio\n(99.9% confidence)', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].set_xlim(0, 90)

# ── Inlier/outlier analysis of our match set ─────────────────────
all_pts1, all_pts2 = extract_match_points(kps1, kps2, good_matches)

# Compute epipolar distances for all matches
pts1_h = np.column_stack([all_pts1, np.ones(len(all_pts1))])
pts2_h = np.column_stack([all_pts2, np.ones(len(all_pts2))])
l2 = (F @ pts1_h.T).T
dist_all = np.abs(np.sum(pts2_h * l2, axis=1)) / (np.sqrt(l2[:,0]**2 + l2[:,1]**2) + 1e-10)

axes[1].hist(dist_all[dist_all < 50], bins=60, color='steelblue', edgecolor='white', lw=0.3)
axes[1].axvline(3.0, color='red', lw=2, ls='--', label='RANSAC threshold = 3px')
inlier_count = (dist_all < 3.0).sum()
axes[1].set_xlabel('Epipolar distance (pixels)')
axes[1].set_ylabel('Match count')
axes[1].set_title(f'Epipolar Distance Distribution\n{inlier_count}/{len(good_matches)} inliers < 3px', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('RANSAC Analysis', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

# Print iteration table
print('RANSAC Iterations Required (99.9% confidence):')
print(f'{"Outlier %":<12}', end='')
for label in labels:
    print(f'{label[:12]:<15}', end='')
print()
print('-' * 72)
for e in [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70]:
    print(f'{e*100:>8.0f}%   ', end='')
    for s in sample_sizes:
        print(f'{ransac_iterations(e,s):<15}', end='')
    print()

## 8. Camera Pose Recovery — Decomposing E into R and t

Given $\mathbf{E}$, we recover the relative camera pose $[\mathbf{R} | \mathbf{t}]$ via SVD:

$$\mathbf{E} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T$$

There are **four candidate solutions** for $(\mathbf{R}, \mathbf{t})$:

$$\mathbf{R}_1 = \mathbf{U}\mathbf{W}\mathbf{V}^T, \quad \mathbf{R}_2 = \mathbf{U}\mathbf{W}^T\mathbf{V}^T$$
$$\mathbf{t}_1 = +\mathbf{u}_3, \quad \mathbf{t}_2 = -\mathbf{u}_3$$

where $\mathbf{W} = \begin{bmatrix} 0 & -1 & 0 \\ 1 & 0 & 0 \\ 0 & 0 & 1 \end{bmatrix}$ and $\mathbf{u}_3$ is the third column of $\mathbf{U}$.

### Cheirality Check

The correct solution is the one where **triangulated 3D points lie in front of both cameras** (positive depth). This is called the **cheirality constraint**.

In [ ]:
def decompose_essential(E):
    """
    Manually decompose E into four (R, t) candidate solutions.

    Returns list of 4 tuples: [(R1,t1), (R1,t2), (R2,t1), (R2,t2)]
    """
    U, sv, Vt = np.linalg.svd(E)

    # Enforce E has exactly two equal singular values
    s = (sv[0] + sv[1]) / 2.0
    E_clean = U @ np.diag([s, s, 0]) @ Vt
    U, _, Vt = np.linalg.svd(E_clean)

    # Ensure proper rotation (det = +1)
    if np.linalg.det(U) < 0: U  *= -1
    if np.linalg.det(Vt) < 0: Vt *= -1

    W = np.array([[0,-1, 0],
                  [1, 0, 0],
                  [0, 0, 1]], dtype=np.float64)

    R1 = U @ W   @ Vt
    R2 = U @ W.T @ Vt
    t  = U[:, 2]     # translation (up to sign)

    return [(R1,  t), (R1, -t), (R2,  t), (R2, -t)]


def triangulate_point(R, t, K, p1, p2):
    """
    Triangulate a single 3D point from two image correspondences.
    Uses the Direct Linear Transform (DLT) method.
    """
    P1 = K @ np.hstack([np.eye(3),    np.zeros((3,1))])   # camera 1 at origin
    P2 = K @ np.hstack([R, t.reshape(3,1)])                # camera 2

    # Build 4×4 system Ax = 0
    A = np.array([
        p1[1]*P1[2] - P1[1],
        P1[0] - p1[0]*P1[2],
        p2[1]*P2[2] - P2[1],
        P2[0] - p2[0]*P2[2]
    ])
    _, _, Vt = np.linalg.svd(A)
    X = Vt[-1]
    return X[:3] / X[3]   # dehomogenise


def cheirality_check(R, t, K, pts1, pts2):
    """
    Count how many triangulated points are in front of both cameras.
    The correct (R, t) maximises this count.
    """
    P1 = K @ np.hstack([np.eye(3), np.zeros((3,1))])
    P2 = K @ np.hstack([R, t.reshape(3,1)])

    pts1_h = pts1.T  # (2, N)
    pts2_h = pts2.T

    pts4d = cv2.triangulatePoints(P1, P2, pts1_h, pts2_h)  # (4, N)
    pts3d = (pts4d[:3] / pts4d[3]).T                       # (N, 3)

    # Check depth in camera 1 (Z > 0)
    depth1 = pts3d[:, 2]

    # Check depth in camera 2
    X_cam2 = (R @ pts3d.T + t.reshape(3,1)).T
    depth2 = X_cam2[:, 2]

    n_positive = np.sum((depth1 > 0) & (depth2 > 0))
    return n_positive, pts3d


# ── Recover camera pose ──────────────────────────────────────────
candidates = decompose_essential(E)

print('Evaluating 4 candidate (R, t) solutions via cheirality check...')
print(f'{"Solution":<12} {"Positive depths":>18} {"Selected":>10}')
print('-' * 44)

best_count = -1
best_R, best_t, best_pts3d = None, None, None

# Use subset of inliers for speed
n_check = min(200, len(pts1_e))
idx_check = np.random.choice(len(pts1_e), n_check, replace=False)

for i, (R_c, t_c) in enumerate(candidates):
    count, pts3d = cheirality_check(R_c, t_c, K, pts1_e[idx_check], pts2_e[idx_check])
    selected = ' ◄ BEST' if count > best_count else ''
    print(f'Solution {i+1:<3} {count:>18} {selected}')
    if count > best_count:
        best_count = count
        best_R, best_t = R_c.copy(), t_c.copy()
        best_pts3d = pts3d

# ── Also use OpenCV's recoverPose (equivalent, more numerically stable)
_, R_cv, t_cv, mask_cv = cv2.recoverPose(E, pts1_e, pts2_e, K)

print(f'\nRecovered rotation R (OpenCV):')
print(np.round(R_cv, 4))
rvec, _ = cv2.Rodrigues(R_cv)
angle   = np.linalg.norm(rvec) * 180 / np.pi
print(f'Rotation angle: {angle:.2f}°')
print(f'\nRecovered translation t (unit vector):')
print(np.round(t_cv.ravel(), 4))

R_final = R_cv
t_final = t_cv.ravel()

## 9. Triangulation

Given the recovered camera poses $[\mathbf{R}_1|\mathbf{t}_1]$ and $[\mathbf{R}_2|\mathbf{t}_2]$ and matched 2D points, triangulation recovers the 3D position of each scene point.

### Direct Linear Transform (DLT)

For each 3D point $\mathbf{X}$, we have two projection equations (one per camera):

$$\lambda_1 \mathbf{x}_1 = \mathbf{P}_1 \mathbf{X}, \quad \lambda_2 \mathbf{x}_2 = \mathbf{P}_2 \mathbf{X}$$

Eliminating the scale factors $\lambda$ gives a homogeneous system $\mathbf{A}\mathbf{X} = 0$, solved via SVD:

$$\mathbf{A} = \begin{bmatrix}
x_1 \mathbf{p}_3^{1T} - \mathbf{p}_1^{1T} \\
y_1 \mathbf{p}_3^{1T} - \mathbf{p}_2^{1T} \\
x_2 \mathbf{p}_3^{2T} - \mathbf{p}_1^{2T} \\
y_2 \mathbf{p}_3^{2T} - \mathbf{p}_2^{2T}
\end{bmatrix}$$

The 3D point is the right singular vector corresponding to the smallest singular value.

### Triangulation Angle

Accuracy improves with a **larger baseline** (angle between viewing rays). Too small an angle (nearly parallel rays) leads to poorly conditioned triangulation and large depth errors.

In [ ]:
def triangulate_all(R1, t1, R2, t2, K, pts1, pts2):
    """
    Triangulate all matched point pairs using cv2.triangulatePoints.

    Camera 1 is placed at world origin: P1 = K [I | 0]
    Camera 2 is displaced:              P2 = K [R | t]

    Returns:
        pts3d : (N, 3) float64 world coordinates
        depths: (N,)   depth values in camera 1 frame
    """
    P1 = K @ np.hstack([R1, t1.reshape(3,1)])
    P2 = K @ np.hstack([R2, t2.reshape(3,1)])

    pts4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)  # (4, N)
    pts3d = (pts4d[:3] / pts4d[3]).T                        # (N, 3)
    depths = pts4d[2] / pts4d[3]                            # depth in P1 frame

    return pts3d, depths


def triangulation_angle(pts3d, c1, c2):
    """
    Compute the triangulation angle for each 3D point.
    This is the angle at the 3D point between the two rays from c1 and c2.
    """
    r1 = pts3d - c1   # rays from camera 1
    r2 = pts3d - c2   # rays from camera 2
    cos_angle = np.sum(r1 * r2, axis=1) / (
        np.linalg.norm(r1, axis=1) * np.linalg.norm(r2, axis=1) + 1e-10
    )
    return np.degrees(np.arccos(np.clip(cos_angle, -1, 1)))


# Camera 1 at origin
R1 = np.eye(3)
t1 = np.zeros(3)
R2 = R_final
t2 = t_final

# Triangulate all inlier matches
pts3d_all, depths_all = triangulate_all(R1, t1, R2, t2, K, pts1_e, pts2_e)

# Filter: keep points in front of both cameras with reasonable depth
X_cam2  = (R2 @ pts3d_all.T + t2.reshape(3,1)).T
depth2  = X_cam2[:, 2]
valid   = (depths_all > 0) & (depth2 > 0) & (depths_all < np.percentile(depths_all[depths_all>0], 98))
pts3d   = pts3d_all[valid]

print(f'Triangulated {len(pts3d_all)} points, {valid.sum()} pass depth filter')
print(f'Depth range  : [{depths_all[valid].min():.3f}, {depths_all[valid].max():.3f}]')

# Camera centres
c1 = -R1.T @ t1
c2 = -R2.T @ t2

angles = triangulation_angle(pts3d, c1, c2)
print(f'Triangulation angles: mean={angles.mean():.2f}°  median={np.median(angles):.2f}°')

# ── Reprojection error ────────────────────────────────────────────
def reprojection_error(pts3d, K, R, t, pts2d_obs):
    """Mean reprojection error in pixels."""
    proj, _ = project_points(pts3d, K, R, t)
    err = np.linalg.norm(proj - pts2d_obs, axis=1)
    return err.mean(), err

err1_mean, err1 = reprojection_error(pts3d, K, R1, t1, pts1_e[valid])
err2_mean, err2 = reprojection_error(pts3d, K, R2, t2, pts2_e[valid])
print(f'\nReprojection error — Camera 1: {err1_mean:.3f} px')
print(f'Reprojection error — Camera 2: {err2_mean:.3f} px')

# ── Visualise reprojection ────────────────────────────────────────
proj1, _ = project_points(pts3d, K, R1, t1)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

axes[0].imshow(img1_rgb)
obs = pts1_e[valid]
axes[0].scatter(obs[:,0], obs[:,1], c='lime', s=8, label='Observed', zorder=3)
axes[0].scatter(proj1[:,0], proj1[:,1], c='red', s=4, marker='+', label=f'Reprojected ({err1_mean:.2f}px)', zorder=4)
for o, p in zip(obs[:80], proj1[:80]):
    axes[0].plot([o[0],p[0]], [o[1],p[1]], 'y-', lw=0.5, alpha=0.6)
axes[0].set_title(f'Reprojection into Camera 1\nMean error: {err1_mean:.2f} px', fontweight='bold')
axes[0].legend(loc='upper right', fontsize=8); axes[0].axis('off')

axes[1].hist(np.concatenate([err1, err2]), bins=40, color='steelblue', edgecolor='white', lw=0.5)
axes[1].axvline(err1_mean, color='green', lw=2, ls='--', label=f'Cam1 mean: {err1_mean:.2f}px')
axes[1].axvline(err2_mean, color='red',   lw=2, ls='--', label=f'Cam2 mean: {err2_mean:.2f}px')
axes[1].set_xlabel('Reprojection error (pixels)')
axes[1].set_ylabel('Count')
axes[1].set_title('Reprojection Error Distribution', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('Triangulation Results', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Incremental SfM — More Than Two Views

Real SfM extends to **N cameras** incrementally:

```
1. Initialise with the best two-view pair (largest baseline + most inliers)
2. For each new image:
   a. Find 2D-3D correspondences (existing 3D points visible in new image)
   b. Estimate new camera pose via PnP (Perspective-n-Point)
   c. Triangulate new point pairs not yet in the 3D model
   d. Run bundle adjustment to reduce accumulated drift
3. Repeat until all images are registered
```

### PnP — Pose from 2D-3D Correspondences

Given $N$ known 3D points and their 2D projections in a new view, **PnP** solves for the camera pose $[\mathbf{R}|\mathbf{t}]$ directly. OpenCV's `solvePnPRansac` uses the efficient **EPnP** or **iterative** algorithm.

In [ ]:
def make_multiview_sequence(base_img, n_views=5, seed=1):
    """
    Synthesise a sequence of N views with gradually changing pose.
    Returns list of (image, H_world_to_cam) tuples.
    """
    rng   = np.random.default_rng(seed)
    views = []
    for i in range(n_views):
        angle = rng.uniform(-20, 20)
        scale = rng.uniform(0.85, 1.1)
        tx    = rng.uniform(-60, 60)
        ty    = rng.uniform(-40, 40)
        img_i, _, H = make_view_pair(base_img, angle_deg=angle,
                                       scale=scale, tx=int(tx), ty=int(ty))
        views.append((img_i, H))
    return views


class IncrementalSfM:
    """
    Minimal incremental SfM pipeline for educational purposes.
    Handles: feature detection, pairwise matching, essential matrix,
    triangulation, PnP registration, and iterative refinement.
    """

    def __init__(self, K, n_features=1500, ratio_thresh=0.75):
        self.K             = K
        self.n_features    = n_features
        self.ratio_thresh  = ratio_thresh

        # State
        self.images        = []
        self.all_kps       = []
        self.all_descs     = []
        self.all_pts       = []
        self.camera_poses  = {}    # {img_idx: (R, t)}
        self.points3d      = []    # list of (X, Y, Z)
        self.point_colors  = []    # list of (R, G, B)
        self.observations  = {}    # {pt3d_idx: [(img_idx, kp_idx), ...]}

    def add_image(self, img):
        """Register an image and detect its features."""
        kps, descs, pts = detect_features(img, self.n_features)
        idx = len(self.images)
        self.images.append(img)
        self.all_kps.append(kps)
        self.all_descs.append(descs)
        self.all_pts.append(pts)
        return idx

    def match_pair(self, i, j):
        """Match features between images i and j."""
        matches, _ = match_features(
            self.all_descs[i], self.all_descs[j], self.ratio_thresh
        )
        if len(matches) < 8:
            return None, None
        pts_i = np.float32([self.all_kps[i][m.queryIdx].pt for m in matches])
        pts_j = np.float32([self.all_kps[j][m.trainIdx].pt for m in matches])
        return pts_i, pts_j

    def initialise(self, i=0, j=1):
        """Bootstrap the reconstruction from a two-view pair."""
        p_i, p_j = self.match_pair(i, j)
        if p_i is None:
            print('Initialisation failed: not enough matches')
            return False

        E, mask = cv2.findEssentialMat(p_i, p_j, self.K,
                                        method=cv2.RANSAC, prob=0.999, threshold=1.0)
        if E is None: return False

        inliers = mask.ravel().astype(bool)
        _, R, t, pose_mask = cv2.recoverPose(E, p_i[inliers], p_j[inliers], self.K)

        R1, t1 = np.eye(3), np.zeros(3)
        R2, t2 = R, t.ravel()

        self.camera_poses[i] = (R1, t1)
        self.camera_poses[j] = (R2, t2)

        pts3d, depths = triangulate_all(R1, t1, R2, t2, self.K,
                                         p_i[inliers], p_j[inliers])

        valid = (depths > 0) & (depths < np.percentile(depths[depths>0]+1e-6, 99))
        self.points3d   = list(pts3d[valid])

        # Sample colors from image 1
        img_rgb = cv2.cvtColor(self.images[i], cv2.COLOR_BGR2RGB)
        for pt in p_i[inliers][valid]:
            x, y = int(np.clip(pt[0], 0, img_rgb.shape[1]-1)), \
                   int(np.clip(pt[1], 0, img_rgb.shape[0]-1))
            self.point_colors.append(img_rgb[y, x] / 255.0)

        print(f'Initialised with images {i}+{j}: {len(self.points3d)} 3D points')
        return True

    def register_image(self, new_idx, ref_idx):
        """Register a new image using PnP against the existing 3D model."""
        if new_idx in self.camera_poses:
            return True   # already registered

        p_ref, p_new = self.match_pair(ref_idx, new_idx)
        if p_ref is None or len(self.points3d) < 6:
            print(f'  Image {new_idx}: skipping — insufficient matches or 3D points')
            return False

        # For this simplified demo, we use the triangulated points
        # and find correspondences via nearest-neighbour matching in 2D
        n_pts = min(len(self.points3d), 300)
        pts3d_arr = np.array(self.points3d[:n_pts], dtype=np.float32)

        R_ref, t_ref = self.camera_poses[ref_idx]
        proj_ref, depths_ref = project_points(
            pts3d_arr, self.K, R_ref, t_ref
        )

        # Find 2D-3D correspondences via proximity
        valid_proj = (depths_ref > 0) & \
                     (proj_ref[:,0] >= 0) & (proj_ref[:,0] < self.images[ref_idx].shape[1]) & \
                     (proj_ref[:,1] >= 0) & (proj_ref[:,1] < self.images[ref_idx].shape[0])

        if valid_proj.sum() < 6:
            return False

        pts3d_valid  = pts3d_arr[valid_proj]
        proj_valid   = proj_ref[valid_proj]

        # Match: for each 3D point projected into ref, find closest match in new image
        # Use FLANN on the matched region
        correspondences_3d, correspondences_2d = [], []
        for k, (px_ref, pt3) in enumerate(zip(proj_valid, pts3d_valid)):
            dists = np.linalg.norm(p_ref - px_ref, axis=1)
            idx_near = np.argmin(dists)
            if dists[idx_near] < 5.0:
                correspondences_3d.append(pt3)
                correspondences_2d.append(p_new[idx_near])

        if len(correspondences_3d) < 6:
            print(f'  Image {new_idx}: only {len(correspondences_3d)} 2D-3D correspondences')
            return False

        pts3d_pnp = np.array(correspondences_3d, dtype=np.float32)
        pts2d_pnp = np.array(correspondences_2d, dtype=np.float32)

        success, rvec, tvec, pnp_inliers = cv2.solvePnPRansac(
            pts3d_pnp, pts2d_pnp, self.K, None,
            confidence=0.99, reprojectionError=4.0,
            flags=cv2.SOLVEPNP_EPNP
        )

        if not success or pnp_inliers is None:
            print(f'  Image {new_idx}: PnP failed')
            return False

        R_new, _ = cv2.Rodrigues(rvec)
        t_new    = tvec.ravel()
        self.camera_poses[new_idx] = (R_new, t_new)

        # Triangulate new points
        R_ref_cam, t_ref_cam = self.camera_poses[ref_idx]
        pts3d_new, depths_new = triangulate_all(
            R_ref_cam, t_ref_cam, R_new, t_new,
            self.K, p_ref, p_new
        )

        img_rgb = cv2.cvtColor(self.images[new_idx], cv2.COLOR_BGR2RGB)
        n_added = 0
        for k, (pt, d) in enumerate(zip(pts3d_new, depths_new)):
            if d > 0 and d < 200:
                self.points3d.append(pt)
                px = p_new[k]
                x  = int(np.clip(px[0], 0, img_rgb.shape[1]-1))
                y  = int(np.clip(px[1], 0, img_rgb.shape[0]-1))
                self.point_colors.append(img_rgb[y, x] / 255.0)
                n_added += 1

        print(f'  Image {new_idx}: pose recovered, +{n_added} new 3D points '
              f'(total: {len(self.points3d)})')
        return True


# ── Run the pipeline on a synthetic sequence ─────────────────────
print('Building synthetic multi-view sequence...')
views = make_multiview_sequence(base_img, n_views=5, seed=42)

sfm = IncrementalSfM(K, n_features=1500)

for img_view, _ in views:
    sfm.add_image(img_view)

print('\nRunning incremental SfM...')
sfm.initialise(0, 1)
for i in range(2, len(views)):
    sfm.register_image(i, i-1)

print(f'\nFinal reconstruction:')
print(f'  Registered cameras : {len(sfm.camera_poses)}')
print(f'  3D points          : {len(sfm.points3d)}')

## 11. Bundle Adjustment

After incremental registration, small errors in each step **accumulate** (drift). Bundle Adjustment (BA) is the global non-linear optimisation that simultaneously refines all camera poses and 3D point positions by minimising the total **reprojection error**:

$$\min_{\{\mathbf{R}_i, \mathbf{t}_i, \mathbf{X}_j\}} \sum_{i,j} \rho\left(\left\| \mathbf{x}_{ij} - \pi(\mathbf{K}, \mathbf{R}_i, \mathbf{t}_i, \mathbf{X}_j) \right\|^2\right)$$

where:
- $\mathbf{x}_{ij}$ is the **observed** 2D location of point $j$ in image $i$
- $\pi(\cdot)$ is the **projection** function
- $\rho(\cdot)$ is a **robust loss** (Huber/Cauchy) to down-weight outliers

### Parameterisation

| Variable | Representation | DOF |
|---|---|---|
| Rotation $\mathbf{R}_i$ | Rodrigues vector (axis-angle) | 3 |
| Translation $\mathbf{t}_i$ | 3D vector | 3 |
| 3D point $\mathbf{X}_j$ | 3D Euclidean | 3 |

### Solving with Levenberg-Marquardt

The Jacobian is **sparse** — each 3D point only appears in a small subset of images. The **Schur complement trick** exploits this structure to solve the normal equations efficiently.

In [ ]:
def rodrigues_to_R(rvec):
    """Convert Rodrigues (axis-angle) vector to rotation matrix."""
    R, _ = cv2.Rodrigues(rvec.reshape(3,1))
    return R


def R_to_rodrigues(R):
    """Convert rotation matrix to Rodrigues vector."""
    rvec, _ = cv2.Rodrigues(R)
    return rvec.ravel()


def pack_params(camera_poses, points3d):
    """
    Pack all optimisation variables into a 1D parameter vector.
    Format: [rvec0, t0, rvec1, t1, ..., X0, X1, ...]
    Each camera: 6 params (3 rotation + 3 translation)
    Each point : 3 params
    """
    params = []
    for R, t in camera_poses:
        params.extend(R_to_rodrigues(R))   # 3
        params.extend(t)                   # 3
    for pt in points3d:
        params.extend(pt)                  # 3
    return np.array(params, dtype=np.float64)


def unpack_params(params, n_cameras, n_points):
    """Unpack 1D parameter vector back into camera poses and 3D points."""
    cameras = []
    for i in range(n_cameras):
        rvec = params[i*6 : i*6+3]
        t    = params[i*6+3 : i*6+6]
        cameras.append((rodrigues_to_R(rvec), t))

    start = n_cameras * 6
    pts3d = params[start:].reshape(n_points, 3)
    return cameras, pts3d


def reprojection_residuals(params, K, n_cameras, n_points,
                            cam_indices, pt_indices, observed_pts):
    """
    Residual function for bundle adjustment.
    Returns flattened (u_proj - u_obs, v_proj - v_obs) for all observations.
    """
    cameras, pts3d = unpack_params(params, n_cameras, n_points)

    residuals = []
    for cam_idx, pt_idx, obs in zip(cam_indices, pt_indices, observed_pts):
        R, t = cameras[cam_idx]
        X = pts3d[pt_idx]

        # Project X
        X_cam = R @ X + t
        if abs(X_cam[2]) < 1e-6:
            residuals.extend([0.0, 0.0])
            continue
        u = K[0,0] * X_cam[0] / X_cam[2] + K[0,2]
        v = K[1,1] * X_cam[1] / X_cam[2] + K[1,2]
        residuals.extend([u - obs[0], v - obs[1]])

    return np.array(residuals)


def bundle_adjustment(K, camera_poses_list, pts3d_arr,
                       cam_indices, pt_indices, observed_pts,
                       max_nfev=200, verbose=0):
    """
    Run sparse bundle adjustment using scipy least_squares.

    Args:
        camera_poses_list : list of (R, t) for each registered camera
        pts3d_arr         : (M, 3) initial 3D point positions
        cam_indices       : (N,) camera index for each observation
        pt_indices        : (N,) 3D point index for each observation
        observed_pts      : (N, 2) observed 2D positions

    Returns:
        cameras_opt : list of refined (R, t)
        pts3d_opt   : (M, 3) refined 3D positions
        result      : scipy OptimizeResult
    """
    n_cameras = len(camera_poses_list)
    n_points  = len(pts3d_arr)

    x0 = pack_params(camera_poses_list, pts3d_arr)

    # Sparse Jacobian structure
    n_obs = len(cam_indices)
    A = lil_matrix((2 * n_obs, 6*n_cameras + 3*n_points), dtype=int)
    for k, (ci, pi) in enumerate(zip(cam_indices, pt_indices)):
        A[2*k,   ci*6   : ci*6+6] = 1
        A[2*k+1, ci*6   : ci*6+6] = 1
        A[2*k,   6*n_cameras + pi*3 : 6*n_cameras + pi*3+3] = 1
        A[2*k+1, 6*n_cameras + pi*3 : 6*n_cameras + pi*3+3] = 1

    result = least_squares(
        reprojection_residuals, x0,
        jac_sparsity=A,
        method='trf',
        loss='huber',             # robust loss
        f_scale=2.0,              # Huber threshold in pixels
        max_nfev=max_nfev,
        verbose=verbose,
        args=(K, n_cameras, n_points, cam_indices, pt_indices, observed_pts)
    )

    cameras_opt, pts3d_opt = unpack_params(result.x, n_cameras, n_points)
    return cameras_opt, pts3d_opt, result


# ── Run BA on the two-camera case ────────────────────────────────
print('Running Bundle Adjustment...')

# Build observation lists from the inlier matches
n_pts_ba = min(len(pts3d), 300)
pts3d_ba = np.array(pts3d[:n_pts_ba])
obs1_ba  = pts1_e[valid][:n_pts_ba]
obs2_ba  = pts2_e[valid][:n_pts_ba]

cam_idx  = np.concatenate([np.zeros(n_pts_ba, int), np.ones(n_pts_ba, int)])
pt_idx   = np.concatenate([np.arange(n_pts_ba), np.arange(n_pts_ba)])
obs_all  = np.vstack([obs1_ba, obs2_ba])

poses_before = [(R1, t1), (R_final, t_final)]

# Compute reprojection error before BA
r_before = reprojection_residuals(
    pack_params(poses_before, pts3d_ba),
    K, 2, n_pts_ba, cam_idx, pt_idx, obs_all
)
err_before = np.sqrt(np.mean(r_before.reshape(-1,2)**2, axis=1)).mean()

cameras_opt, pts3d_opt, ba_result = bundle_adjustment(
    K, poses_before, pts3d_ba,
    cam_idx, pt_idx, obs_all,
    max_nfev=100, verbose=0
)

# Compute reprojection error after BA
r_after = reprojection_residuals(
    pack_params(cameras_opt, pts3d_opt),
    K, 2, n_pts_ba, cam_idx, pt_idx, obs_all
)
err_after = np.sqrt(np.mean(r_after.reshape(-1,2)**2, axis=1)).mean()

print(f'Mean reprojection error — before BA : {err_before:.4f} px')
print(f'Mean reprojection error — after  BA : {err_after:.4f} px')
print(f'Improvement: {100*(err_before - err_after)/err_before:.1f}%')
print(f'BA iterations: {ba_result.nfev}')
print(f'Termination  : {ba_result.message}')

# Plot convergence
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

res_before = np.sqrt(r_before.reshape(-1,2)[:,0]**2 + r_before.reshape(-1,2)[:,1]**2)
res_after  = np.sqrt(r_after.reshape(-1,2)[:,0]**2  + r_after.reshape(-1,2)[:,1]**2)

axes[0].hist(res_before, bins=40, alpha=0.6, color='red',  label=f'Before BA (μ={err_before:.2f}px)')
axes[0].hist(res_after,  bins=40, alpha=0.6, color='blue', label=f'After  BA (μ={err_after:.2f}px)')
axes[0].set_xlabel('Reprojection error (px)')
axes[0].set_ylabel('Count')
axes[0].set_title('Bundle Adjustment — Error Distribution', fontweight='bold')
axes[0].legend(); axes[0].grid(alpha=0.3)

shift3d = np.linalg.norm(pts3d_opt - pts3d_ba, axis=1)
axes[1].hist(shift3d, bins=40, color='steelblue', edgecolor='white', lw=0.5)
axes[1].set_xlabel('3D point shift magnitude')
axes[1].set_ylabel('Count')
axes[1].set_title(f'BA 3D Point Adjustments\nMean shift: {shift3d.mean():.4f}', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 12. 3D Point Cloud Visualization

We now combine everything to visualise the final sparse reconstruction: the 3D point cloud and the recovered camera poses.

In [ ]:
def draw_camera_frustum(fig, R, t, K, H=480, W=640, scale=0.3, color='blue', name='Camera'):
    """
    Add a camera frustum to a Plotly 3D figure.
    The frustum shows the camera's position and orientation in 3D.
    """
    # Camera centre in world coords
    C = -R.T @ t

    # Four corners of the image plane (in camera coordinates)
    corners_cam = np.array([
        [(0 - K[0,2]) / K[0,0], (0 - K[1,2]) / K[1,1], 1],
        [(W - K[0,2]) / K[0,0], (0 - K[1,2]) / K[1,1], 1],
        [(W - K[0,2]) / K[0,0], (H - K[1,2]) / K[1,1], 1],
        [(0 - K[0,2]) / K[0,0], (H - K[1,2]) / K[1,1], 1],
    ]) * scale

    # Transform to world
    corners_world = (R.T @ corners_cam.T - R.T @ t.reshape(3,1)).T

    # Edges of frustum
    edges = []
    for corner in corners_world:
        edges += [C.tolist(), corner.tolist(), [None,None,None]]
    for i in range(4):
        edges += [corners_world[i].tolist(), corners_world[(i+1)%4].tolist(), [None,None,None]]

    ex = [e[0] for e in edges]
    ey = [e[1] for e in edges]
    ez = [e[2] for e in edges]

    fig.add_trace(go.Scatter3d(
        x=ex, y=ey, z=ez,
        mode='lines',
        line=dict(color=color, width=2),
        name=name,
        showlegend=True
    ))

    # Camera centre marker
    fig.add_trace(go.Scatter3d(
        x=[C[0]], y=[C[1]], z=[C[2]],
        mode='markers',
        marker=dict(size=6, color=color),
        name=f'{name} centre',
        showlegend=False
    ))


# ── Interactive Plotly 3D visualisation ───────────────────────────
pts3d_vis   = np.array(pts3d_opt if len(pts3d_opt) > 0 else pts3d)
colors_vis  = np.array(sfm.point_colors[:len(pts3d_vis)]) if sfm.point_colors else None

# Remove outliers using IQR
def remove_outliers_iqr(pts, factor=3.0):
    med = np.median(pts, axis=0)
    mad = np.median(np.abs(pts - med), axis=0) + 1e-6
    mask = np.all(np.abs(pts - med) < factor * mad * 1.4826, axis=1)
    return mask

mask_ok     = remove_outliers_iqr(pts3d_vis)
pts3d_clean = pts3d_vis[mask_ok]

if colors_vis is not None and len(colors_vis) == len(mask_ok):
    colors_clean = colors_vis[mask_ok]
    color_strs = [f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
                  for c in colors_clean]
else:
    depths_clean = pts3d_clean[:, 2]
    d_norm = (depths_clean - depths_clean.min()) / (depths_clean.ptp() + 1e-6)
    cmap   = plt.cm.plasma
    color_strs = [f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
                  for c in cmap(d_norm)[:, :3]]

fig = go.Figure()

# Point cloud
fig.add_trace(go.Scatter3d(
    x=pts3d_clean[:,0],
    y=pts3d_clean[:,1],
    z=pts3d_clean[:,2],
    mode='markers',
    marker=dict(size=2, color=color_strs, opacity=0.8),
    name=f'3D Points ({len(pts3d_clean)})'
))

# Camera frustums
cam_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']
for i, (R_c, t_c) in enumerate(cameras_opt):
    draw_camera_frustum(fig, R_c, t_c, K, scale=0.4,
                         color=cam_colors[i % len(cam_colors)],
                         name=f'Camera {i}')

# Also show SfM cameras if registered
for cam_idx, (R_c, t_c) in list(sfm.camera_poses.items())[2:]:
    draw_camera_frustum(fig, R_c, t_c, K, scale=0.4,
                         color=cam_colors[cam_idx % len(cam_colors)],
                         name=f'SfM Camera {cam_idx}')

fig.update_layout(
    title=dict(text='Structure from Motion — 3D Reconstruction', font=dict(size=15)),
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z (depth)',
        aspectmode='data',
        camera=dict(eye=dict(x=0, y=-2, z=0.5))
    ),
    height=700,
    legend=dict(x=0.01, y=0.99)
)

fig.show()

print(f'\n3D Reconstruction Summary:')
print(f'  Points in cloud      : {len(pts3d_clean)}')
print(f'  Cameras reconstructed: {len(cameras_opt)}')
print(f'  Point cloud extent   : X=[{pts3d_clean[:,0].min():.2f}, {pts3d_clean[:,0].max():.2f}]')
print(f'                         Y=[{pts3d_clean[:,1].min():.2f}, {pts3d_clean[:,1].max():.2f}]')
print(f'                         Z=[{pts3d_clean[:,2].min():.2f}, {pts3d_clean[:,2].max():.2f}]')

In [ ]:
# ── Static matplotlib summary figure ─────────────────────────────

fig = plt.figure(figsize=(18, 12))
gs  = gridspec.GridSpec(2, 3, figure=fig)

# (0,0) — Image 1 with keypoints
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(visualise_keypoints(img1, kps1, max_show=300))
ax1.set_title(f'1. SIFT Keypoints\n({len(kps1)} detected)', fontweight='bold'); ax1.axis('off')

# (0,1) — Feature matches
ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(draw_matches_vis(img1, kps1, img2, kps2, good_matches[:60]))
ax2.set_title(f'2. Feature Matches\n({len(good_matches)} after ratio test)', fontweight='bold'); ax2.axis('off')

# (0,2) — Epipolar lines
ax3 = fig.add_subplot(gs[0, 2])
ax3.imshow(epi2)
ax3.set_title('3. Epipolar Lines\n(Fundamental Matrix)', fontweight='bold'); ax3.axis('off')

# (1,0) — Reprojection
ax4 = fig.add_subplot(gs[1, 0])
ax4.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax4.scatter(obs1_ba[:,0], obs1_ba[:,1], c='lime', s=5, label='Observed')
proj_opt, _ = project_points(pts3d_opt, K, *cameras_opt[0])
ax4.scatter(proj_opt[:,0], proj_opt[:,1], c='red', s=3, marker='+', label=f'Reprojected')
ax4.set_title(f'4. Reprojection after BA\n(error={err_after:.2f}px)', fontweight='bold')
ax4.legend(fontsize=7); ax4.axis('off')

# (1,1) — 3D point cloud (top view)
ax5 = fig.add_subplot(gs[1, 1])
sc = ax5.scatter(pts3d_clean[:,0], pts3d_clean[:,2],
                  c=pts3d_clean[:,1], cmap='plasma', s=2, alpha=0.6)
for i, (R_c, t_c) in enumerate(cameras_opt):
    C = -R_c.T @ t_c
    ax5.scatter(C[0], C[2], s=120, marker='^',
                color=cam_colors[i % len(cam_colors)],
                zorder=5, label=f'Cam {i}')
plt.colorbar(sc, ax=ax5, label='Y height')
ax5.set_xlabel('X'); ax5.set_ylabel('Z (depth)')
ax5.set_title('5. 3D Point Cloud (top view)', fontweight='bold')
ax5.legend(fontsize=7); ax5.grid(alpha=0.3)

# (1,2) — 3D side view
ax6 = fig.add_subplot(gs[1, 2], projection='3d')
ax6.scatter(pts3d_clean[::3,0], pts3d_clean[::3,1], pts3d_clean[::3,2],
             c=pts3d_clean[::3,2], cmap='plasma', s=1, alpha=0.5)
for i, (R_c, t_c) in enumerate(cameras_opt):
    C = -R_c.T @ t_c
    ax6.scatter(*C, s=80, marker='^', color=cam_colors[i % len(cam_colors)], zorder=5)
ax6.set_xlabel('X'); ax6.set_ylabel('Y'); ax6.set_zlabel('Z')
ax6.set_title('6. 3D Point Cloud (3D view)', fontweight='bold')

plt.suptitle('Structure from Motion — Complete Pipeline Summary', fontsize=15, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('/content/sfm_summary.png', dpi=130, bbox_inches='tight')
plt.show()

print('Summary figure saved to /content/sfm_summary.png')

## 13. Upload Your Own Images

Run the full SfM pipeline on your own photo sequence.

In [ ]:
from google.colab import files

print('Upload 3 or more images of the SAME scene from DIFFERENT viewpoints.')
print('Tips:')
print('  - Move the camera sideways between shots (good baseline)')
print('  - Keep ~60-80% overlap between consecutive images')
print('  - Avoid motion blur and low texture scenes')

uploaded = files.upload()

if len(uploaded) >= 2:
    user_imgs = []
    for fname in sorted(uploaded.keys()):
        img_u = cv2.imread(fname)
        if img_u is None:
            img_u = cv2.imdecode(np.frombuffer(uploaded[fname], np.uint8), cv2.IMREAD_COLOR)
        # Resize to manageable resolution
        h_u, w_u = img_u.shape[:2]
        scale_u  = min(1.0, 800 / max(h_u, w_u))
        img_u    = cv2.resize(img_u, (int(w_u*scale_u), int(h_u*scale_u)))
        user_imgs.append(img_u)
        print(f'  Loaded: {fname}  {img_u.shape[:2]}')

    # Estimate K from EXIF focal length or use a reasonable default
    H_u, W_u = user_imgs[0].shape[:2]
    K_user   = build_K(fx=max(H_u,W_u)*1.2, fy=max(H_u,W_u)*1.2, cx=W_u/2, cy=H_u/2)
    print(f'\nUsing estimated K (focal={max(H_u,W_u)*1.2:.0f}px):')
    print(K_user)

    # Run SfM
    sfm_user = IncrementalSfM(K_user, n_features=2000)
    for img_u in user_imgs:
        sfm_user.add_image(img_u)

    success = sfm_user.initialise(0, 1)
    if success:
        for i in range(2, len(user_imgs)):
            sfm_user.register_image(i, i-1)

        # Visualise
        pts_u = np.array(sfm_user.points3d)
        if len(pts_u) > 3:
            mask_u = remove_outliers_iqr(pts_u, factor=4.0)
            pts_u_clean = pts_u[mask_u]

            fig_u = go.Figure()
            colors_u = [f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
                        for c in (np.array(sfm_user.point_colors)[mask_u]
                                  if len(sfm_user.point_colors) == len(pts_u)
                                  else np.ones((len(pts_u_clean), 3)) * 0.5)]

            fig_u.add_trace(go.Scatter3d(
                x=pts_u_clean[:,0], y=pts_u_clean[:,1], z=pts_u_clean[:,2],
                mode='markers',
                marker=dict(size=2, color=colors_u, opacity=0.9),
                name=f'3D points ({len(pts_u_clean)})'
            ))
            for i, (R_u, t_u) in sfm_user.camera_poses.items():
                draw_camera_frustum(fig_u, R_u, t_u, K_user, scale=0.5,
                                     color=cam_colors[i % len(cam_colors)],
                                     name=f'Camera {i}')

            fig_u.update_layout(
                title='SfM — Your Images',
                scene=dict(aspectmode='data'),
                height=650
            )
            fig_u.show()
    else:
        print('Initialisation failed. Try images with more overlap and texture.')
else:
    print('Upload at least 2 images to run SfM on your own data.')

## Summary — The Full SfM Pipeline

| Step | Method | Key Equation / Tool |
|---|---|---|
| **Camera model** | Pinhole projection | $\lambda\mathbf{x} = \mathbf{K}[\mathbf{R}|\mathbf{t}]\mathbf{X}$ |
| **Feature detection** | SIFT | DoG scale-space extrema |
| **Feature matching** | FLANN + ratio test | $d_1/d_2 < 0.75$ |
| **Fundamental matrix** | 8-point + RANSAC | $\mathbf{x}'^T\mathbf{F}\mathbf{x} = 0$ |
| **Essential matrix** | 5-point algorithm | $\mathbf{E} = \mathbf{K}'^T\mathbf{F}\mathbf{K}$ |
| **Pose recovery** | SVD + cheirality | $\mathbf{E} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^T \rightarrow (\mathbf{R}, \mathbf{t})$ |
| **Triangulation** | DLT | Solve $\mathbf{A}\mathbf{X}=0$ via SVD |
| **New view registration** | PnP + RANSAC | `cv2.solvePnPRansac` |
| **Bundle adjustment** | Levenberg-Marquardt | Minimise $\sum\|\mathbf{x} - \pi(\mathbf{X})\|^2$ |

### Production SfM Systems
For real applications consider these battle-tested libraries:
- **COLMAP** — https://colmap.github.io  (C++/Python, the state of the art)
- **OpenSfM** — https://github.com/mapillary/OpenSfM  (Python, used by Mapillary)
- **Meshroom** — https://alicevision.org  (GUI-based, open source)
- **OpenMVG** — https://github.com/openMVG/openMVG  (research-oriented)

### Further Reading
- Hartley & Zisserman — *Multiple View Geometry in Computer Vision* (the textbook)
- Snavely et al. — *Photo Tourism* (SIGGRAPH 2006) — origin of modern SfM
- Schönberger & Frahm — *Structure-from-Motion Revisited* (CVPR 2016) — COLMAP paper